In [2]:
#Import required packages
import csv
# import math
import os
import fiona
# from itertools import tee
import pprint
import numpy as np
from shapely.geometry import LineString
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors
import cartopy.crs as ccrs
# import cartopy.io.shapereader as shpreader
import contextily as ctx

In [4]:
unique_link_ids = set()

flows_filename = os.path.join('data','link_use_central_oxford.csv')

flows = []

with open(flows_filename, 'r') as source:
    reader = csv.DictReader(source)
    for item in reader:
        unique_link_ids.add(int(item['edgeID']))
        flows.append({
            'EdgeID': int(item['edgeID']),
            'hour': item['hour'],
            'vehicles': int(item['vehicles']),
        })
        
print(flows[:1])

[{'EdgeID': 47528, 'hour': 'MIDNIGHT', 'vehicles': 16}]


In [3]:
links = {}
    
with fiona.open(os.path.join('data','fullNetworkWithEdgeIDs.shp')) as source:
    for item in source:
        link = int(item['properties']['EdgeID'])
        if link in unique_link_ids:
            geom = LineString(item['geometry']['coordinates'])
            links[link] = {
                'EdgeID': link,
                'geom': geom
            }

print(links)

{73199: {'EdgeID': 73199, 'geom': <shapely.geometry.linestring.LineString object at 0x0000024B1168B6D8>}, 53203: {'EdgeID': 53203, 'geom': <shapely.geometry.linestring.LineString object at 0x0000024B11678DD8>}, 67418: {'EdgeID': 67418, 'geom': <shapely.geometry.linestring.LineString object at 0x0000024B11668F60>}, 60575: {'EdgeID': 60575, 'geom': <shapely.geometry.linestring.LineString object at 0x0000024B1168BBE0>}, 63440: {'EdgeID': 63440, 'geom': <shapely.geometry.linestring.LineString object at 0x0000024B1168BBA8>}, 73222: {'EdgeID': 73222, 'geom': <shapely.geometry.linestring.LineString object at 0x0000024B1168BCC0>}, 52723: {'EdgeID': 52723, 'geom': <shapely.geometry.linestring.LineString object at 0x0000024B1168B9B0>}, 73198: {'EdgeID': 73198, 'geom': <shapely.geometry.linestring.LineString object at 0x0000024B1168B5C0>}, 80522: {'EdgeID': 80522, 'geom': <shapely.geometry.linestring.LineString object at 0x0000024B1168BCF8>}, 55563: {'EdgeID': 55563, 'geom': <shapely.geometry.lin

In [4]:
hourly_maxes = []
for interval_key, interval_name in [
        ('MIDNIGHT', '00'),
        ('ONEAM', '01'),
        ('TWOAM', '02'),
        ('THREEAM', '03'),
        ('FOURAM', '04'),
        ('FIVEAM', '05'),
        ('SIXAM', '06'),
        ('SEVENAM', '07'),
        ('EIGHTAM', '08'),
        ('NINEAM', '09'),
        ('TENAM', '10'),
        ('ELEVENAM', '11'),
        ('NOON', '12'),
        ('ONEPM', '13'),
        ('TWOPM', '14'),
        ('THREEPM', '15'),
        ('FOURPM', '16'),
        ('FIVEPM', '17'),
        ('SIXPM', '18'),
        ('SEVENPM', '19'),
        ('EIGHTPM', '20'),
        ('NINEPM', '21'),
        ('TENPM', '22'),
        ('ELEVENPM', '23')
    ]:
    hourly_maxes.append(max(float(line['vehicles']) for line in flows if line['hour'] == interval_key))
flow_max = max(hourly_maxes)
print(flow_max)

4556.0


In [5]:
for interval_key, interval_name in [
    ('MIDNIGHT', '00'),
    ('ONEAM', '01'),
    ('TWOAM', '02'),
    ('THREEAM', '03'),
    ('FOURAM', '04'),
    ('FIVEAM', '05'),
    ('SIXAM', '06'),
    ('SEVENAM', '07'),
    ('EIGHTAM', '08'),
    ('NINEAM', '09'),
    ('TENAM', '10'),
    ('ELEVENAM', '11'),
    ('NOON', '12'),
    ('ONEPM', '13'),
    ('TWOPM', '14'),
    ('THREEPM', '15'),
    ('FOURPM', '16'),
    ('FIVEPM', '17'),
    ('SIXPM', '18'),
    ('SEVENPM', '19'),
    ('EIGHTPM', '20'),
    ('NINEPM', '21'),
    ('TENPM', '22'),
    ('ELEVENPM', '23')
    ]:
    for line in flows:
        link = int(line['EdgeID'])

        if line['hour'] == interval_key and link in list(unique_link_ids):
            links[link]['flow'] = int(line['vehicles'])
            links[link]['hour'] = interval_key              
    
    def plot_map(interval, links, flow_max, output_filename):

        # Create figure
        plt.figure(figsize=(6, 10), dpi=150)

        # Get projection
        proj = ccrs.OSGB()

        # Create axes to hold plot
        ax = plt.axes([0.025, 0.025, 0.95, 0.93], projection=proj)
        extent = [440000, 460000, 200000, 220000] # xmin xmax ymin ymax for GB
        ax.set_extent(extent, crs=proj)
        ax.outline_patch.set_visible(False)

        ax.text(3000, 1100000, '{}:00'.format(interval), fontsize=16)

        print("Plotted background")
        binwidth = 50
        flows = np.unique([int(link['flow'] / binwidth) for link in links.values() if 'flow' in link]) * binwidth
        print(flows)
        # Choose colormap
        # see https://matplotlib.org/users/colormaps.html for options
        colors = plt.get_cmap('YlOrRd')
        color_map = plt.cm.ScalarMappable(cmap=colors, norm=matplotlib.colors.LogNorm(vmin=0.0001, vmax=flow_max))

        geoms_by_flow = {
            0.0001: []  # Catch-all for zero/negative flows
        }

        for flow in flows:
            if flow == 0:
                flow = 0.0001
            geoms_by_flow[flow] = []

        for link in links.values():
            if 'flow' in link:
                flow = int(link['flow'] / binwidth) * binwidth
                if flow == 0:
                    flow = 0.0001
            else:
                flow = 0.0001
            geoms_by_flow[flow].append(link['geom'])
        print("Grouped geoms")

        for flow in flows:
            if flow <= 0:
                flow = 0.0001
            color = color_map.to_rgba(flow)
            ax.add_geometries(
                geoms_by_flow[flow],
                crs=proj,
                edgecolor=color,
                facecolor='none',
                zorder=2)
        print("Plotted geoms")

        # Add colorbar
        color_map._A = []
        cbar = plt.colorbar(color_map, ax=ax, fraction=0.1, pad=0.03, drawedges=False, orientation='horizontal')
        cbar.outline.set_color("none")
        cbar.ax.set_xlabel('Flow volume')
        print("Plotted colorbar")

        # Save to file
        plt.savefig(output_filename)
        plt.close()

    plot_map(interval_name, links, flow_max, os.path.join('vis', interval_name))
    print("Done", interval_name)

Plotted background
[ 0 50]
Grouped geoms
Plotted geoms
Plotted colorbar
Done 00
Plotted background
[0]
Grouped geoms
Plotted geoms
Plotted colorbar
Done 01
Plotted background
[0]
Grouped geoms
Plotted geoms
Plotted colorbar
Done 02
Plotted background
[0]
Grouped geoms
Plotted geoms
Plotted colorbar
Done 03
Plotted background
[ 0 50]
Grouped geoms
Plotted geoms
Plotted colorbar
Done 04
Plotted background
[  0  50 100 150 200 250 300 350]
Grouped geoms
Plotted geoms
Plotted colorbar
Done 05
Plotted background
[   0   50  100  150  200  250  300  350  400  450  500  550  600  650
  700  750  800  850  900  950 1000]
Grouped geoms
Plotted geoms
Plotted colorbar
Done 06
Plotted background
[  50  250  300  350  400  450  500  550  600  650  700  750  800  850
  900 1000 1050 1150 1250 1300 1450 1500 1650 1800 1950 2000 2050 2100
 2150 2250 2300 2350 2400 2600 2700 2750]
Grouped geoms
Plotted geoms
Plotted colorbar
Done 07
Plotted background
[ 100  150  400  450  550  600  650  700  750  800 

In [6]:
import glob
import imageio

filenames = glob.glob(os.path.join('vis','*.png'))

images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave(os.path.join('movie.gif'), images)